<a href="https://colab.research.google.com/github/malakanton/pet_projects/blob/main/LogRegression_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Logistic Regression from scratch

During my studying, I figured it would be a great experience to create a Logistic Regression on my own to dive deeper into ML math algorythms, python coding and better understand how it works. I've used 2 datasets to test the custom LodRegration I created and basic Sklearn logistic regression. Below you can find the outcome of the work i've done.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

These are the math formulas logistic regression is based on. 

Source: https://academy.yandex.ru/handbook/ml

Loss function for Logistic regression

$$
L(w,X,y)=−∑_i(y_ilog(σ(⟨w,x_i⟩))+(1−y_i)log(σ(−⟨w,x_i⟩)))
$$

Gradient

$$
∇wL(y,X,w)=−∑_ix_i(y_i−σ(⟨w,x_i⟩)))
$$

Model predictions:
$$
p=σ(⟨w,x_i⟩) = {{1}\over{1+e^{−⟨w,x_i⟩}}}
$$

Below you can see the code of my logistic regression class.
Desctiptions of methods I'm using:


*   _ _ init_ _ is a must have method using every time the class is created. It takes two hypreparameters of my logistic regression: lr - learning rate(the gradient descent step), and a th - thereshold(a decision boundary, the output is either 0 or 1)
*   _normalize method is a method conducting the features normalization in order to get all features into range of -1 to 1 by substracting the mean value of each feature and dividing by standard deviation. It takes the raw numeric features and returns the normalized ones.
*   fit method(callable) provides the actual weights fitting according to the features by SGD(stohastic gradient descent). Brief description:
    - Normalization of income features(keep in mind the class will store the mean and std for each feature in order to use it for unknown data)
    - Adding of extra feature - bias (so all weights are stored in one numpy array)
    - For each batch I extract a particular amount of objects to learn on(using batch_size) 
    - Calculate predictions by a sigmoid function
    - Calculating the gradient
    - Do the gradient step opposite side of the gradient direction multiplying by our learning rate
    - Calculate the losses and stack it in a list
    - Calculate the total losses for each epoch
*   predict_proba method is needed to estimate the probabilities for each object in a test sample relating to the class 1 using the sigmoid function
*   predict mothod use the predict_proba method and a threshold in order to predict the class the object is related to
*   _sigma method is an internal class function used to calculate the sigmoid
*   _loss function calculates the loss by BCE(binary cross entropy function). I clip the input array in order to prevent the zeros in log
*   _grad method calculates the gradients using the formula above



Here is a code for the class:

In [ ]:
class LogRegression():

    """Custom Logistic Regression"""
    
    def __init__(self, lr=0.001, th=0.5):
        self.lr = lr
        self.th = th


    def _normalize(self, X): # features normalizing
        X = X.copy()
        self.std = np.zeros(X.shape[1])
        self.mean = np.zeros(X.shape[1])
        for i in range(X.shape[1]): #for each feature
            self.std[i] = X[:,i].std() #calculation standart deviation
            self.mean[i] = X[:,i].mean() #calculation mean
            X[:,i] = (X[:,i]- self.mean[i]) / self.std[i]
        
        return X


    def fit(self, X, y, batch_size=100, iters=100, iter_display=10): # gradient descent learning
        X = X.copy()
        X = self._normalize(X)
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1) #adding additional feature - bias
                
        self.w = np.random.randn(X.shape[1]) #initializing random weights
        losses = []

        for epoch in range(1,iters): # epochs iteration
            for i in range((X.shape[0]-1)//batch_size + 1): #batches iteration
                #batches extraction
                start_i = i * batch_size
                end_i = start_i + batch_size
                X_batch = X[start_i:end_i]
                y_batch = y[start_i:end_i]
                #batch predictions
                predictions = self._sigma(X_batch, self.w) #sigmoid
                gradient = self._grad(X_batch, y_batch, predictions) # calc gradients
                self.w -= self.lr * gradient # gradient descent step
                
                loss = self._loss(y_batch, predictions)#loss calculation
                losses.append(loss)
            total_preds = self._sigma(X, self.w)
            loss = self._loss(y, total_preds)
            print(f'Iteration: {epoch}, Loss: {loss}') #print during learning process 


    def predict_proba(self, X): # predicting scores from 0 to 1
        X_test = X.copy()
        for i in range(X_test.shape[1]): #normalization using mean and std values from train
            X_test[:,i] = (X_test[:,i]- self.mean[i]) / self.std[i]
        
        X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1) #adding bias
                
        return self._sigma(X_test, self.w) #predict = sigmoid function results


    def predict(self, X): # predicting binary classes

        return np.array(self.predict_proba(X)>=self.th).astype(int)


    def _sigma(self, X, w): #sigmoid function for calculation predictions
        logit = np.dot(X,w)
        return 1 / (1 + np.exp(- logit))
        

    def _loss (self, y, preds): # loss function
        preds = np.clip(preds, 1e-10, 1 - 1e-10)
        return -np.sum(y * np.log(preds) + (1 - y) * np.log(1 - preds))
        

    def _grad(self, X, y, preds): # gradient based on features, target and predictions
        w_grad = X.T @ (preds - y)
        return w_grad

##Let's test the model we defined:

## Mobile operator's customers behavior dataset

It contains information about mobile operator users' behavior.
- calls - number of calls,
- minutes - total duration of calls in minutes,
- messages -  number of sms 
- mb_used - consumed internet traffic in Mb
- is_ultra - which tariff was used during the month ("Ultra" — 1, "Smart" — 0)

In [ ]:
url = 'https://code.s3.yandex.net/datasets/users_behavior.csv'
df = pd.read_csv(url)
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


We have 4 numeric features and one binary target - is_ultra

In [ ]:
df.shape

(3214, 5)

Classes are slightly inbalanced

In [ ]:
df.is_ultra.mean()

0.30647168637212197

Extracting features and a target

In [ ]:
X = df.drop('is_ultra', axis=1).values
y = df.is_ultra.values

Train/test selection

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=42, stratify=y)

Learning process

In [ ]:
lr = LogRegression(lr = 0.0005)
lr.fit(X_train,y_train, iters = 25)

Iteration: 1, Loss: 1650.9549083832262
Iteration: 2, Loss: 1594.9481464942287
Iteration: 3, Loss: 1548.2772914945083
Iteration: 4, Loss: 1510.4035149096644
Iteration: 5, Loss: 1480.4819798878298
Iteration: 6, Loss: 1457.4512912757298
Iteration: 7, Loss: 1440.1509285630157
Iteration: 8, Loss: 1427.4362848052992
Iteration: 9, Loss: 1418.2668673329042
Iteration: 10, Loss: 1411.7570865530015
Iteration: 11, Loss: 1407.192263307969
Iteration: 12, Loss: 1404.0198244064495
Iteration: 13, Loss: 1401.8269236783312
Iteration: 14, Loss: 1400.3134127107978
Iteration: 15, Loss: 1399.2657561179105
Iteration: 16, Loss: 1398.534629822835
Iteration: 17, Loss: 1398.0170306290104
Iteration: 18, Loss: 1397.6426768184338
Iteration: 19, Loss: 1397.3640285204458
Iteration: 20, Loss: 1397.149152028311
Iteration: 21, Loss: 1396.9767163605186
Iteration: 22, Loss: 1396.8325370001332
Iteration: 23, Loss: 1396.7072158973642
Iteration: 24, Loss: 1396.5945445717916


Custom logistic regression results

In [ ]:
y_hat_lr = lr.predict(X_test)
print('LogRegression from scratch:')
print('mean', y_hat_lr.mean())
print('accuracy', accuracy_score(y_test, y_hat_lr))
print('ROC-AUC: ', roc_auc_score(y_test, lr.predict_proba(X_test)))

LogRegression from scratch:
mean 0.08955223880597014
accuracy 0.7338308457711443
ROC-AUC:  0.6730701984439199


Sklearn logistic regression

In [ ]:
lr_skl = LogisticRegression(solver='saga')
lr_skl.fit(X_train,y_train)
y_hat_skl = lr_skl.predict(X_test)
print('sklearn LogRegression:')
print('mean', y_hat_skl.mean())
print('accuracy', accuracy_score(y_test, y_hat_skl))
print('ROC-AUC: ',roc_auc_score(y_test, lr_skl.predict_proba(X_test)[:,0]))

sklearn LogRegression:
mean 0.0
accuracy 0.6940298507462687
ROC-AUC:  0.6322303814435993


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


While our classes are quite imbalanced, accuracy is not the finest way to evalueate the models. The ROC-AUC score which is not dependent on the threshold is the better way. But even so our custom logistic regression is a bit better than the one from sklearn.

## Bank clients churn dataset

I've already done the data preprocessing for this dataset to get it suitable for the logistic regression(missed values input, categorical features encoding etc.), so we skip this step here.

Data description:

- credit_score — credit rating
- gender — gender
- age — age
- tenure — how many years a person has been a customer of the bank
- balance — account balance
- num_of_products — the number of bank products used by the customer
- has_cr_card — availability of a credit card
- is_active — client activity
- estimated_salary — estimated salary
- exited — the fact of the client's departure
- germany - country of residence
- spain - country of residence

In [ ]:
!gdown 1jnjIh-kZWpoUqc59yj9XGDH_iBZjj8ab

Downloading...
From: https://drive.google.com/uc?id=1jnjIh-kZWpoUqc59yj9XGDH_iBZjj8ab
To: /content/bank_churn_prepared.csv
100% 514k/514k [00:00<00:00, 105MB/s]


In [ ]:
df_bank = pd.read_csv('/content/bank_churn_prepared.csv', index_col=0)
df_bank.head()

,credit_score,gender,age,tenure,balance,num_of_products,has_cr_card,is_active,estimated_salary,exited,germany,spain
0,619,0,42,0.2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,0.1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,0.8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,0.1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,0.2,125510.82,1,1,1,79084.10,0,0,1


We have 11 numeric features and one binary target - "exited"

In [ ]:
df_bank.shape

(10000, 12)

In [ ]:
df_bank.exited.mean()

0.2037

This one is also unbalanced 20/80

Features and target extraction, train/test selection

In [ ]:
X_b = df_bank.drop('is_active', axis=1).values
y_b = df_bank.is_active.values

X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(X_b, y_b, test_size=0.25, random_state=42, stratify=y_b)

Fitting process

In [ ]:
clf = LogRegression(lr = 0.0005)
clf.fit(X_b_train, y_b_train, iters = 20)

Iteration: 1, Loss: 6558.340938369177
Iteration: 2, Loss: 5393.629662095283
Iteration: 3, Loss: 5105.07392611429
Iteration: 4, Loss: 5040.332352493768
Iteration: 5, Loss: 5025.83706801451
Iteration: 6, Loss: 5022.520214738344
Iteration: 7, Loss: 5021.743007330459
Iteration: 8, Loss: 5021.5570447458385
Iteration: 9, Loss: 5021.511008693819
Iteration: 10, Loss: 5021.498347572343
Iteration: 11, Loss: 5021.493784656097
Iteration: 12, Loss: 5021.4913863912125
Iteration: 13, Loss: 5021.489774872824
Iteration: 14, Loss: 5021.488606021512
Iteration: 15, Loss: 5021.48775607409
Iteration: 16, Loss: 5021.487148022863
Iteration: 17, Loss: 5021.4867206338295
Iteration: 18, Loss: 5021.48642465966
Iteration: 19, Loss: 5021.486222077396


In [ ]:
y_hat_lr = clf.predict(X_b_test)
print('LogRegression from scratch:')
print('mean', y_hat_lr.mean())
print('accuracy', accuracy_score(y_b_test, y_hat_lr))
print('ROC-AUC: ', roc_auc_score(y_b_test, clf.predict_proba(X_b_test)))

LogRegression from scratch:
mean 0.6192
accuracy 0.5576
ROC-AUC:  0.5982360658426091


In [ ]:
skl_clf = LogisticRegression()

skl_clf.fit(X_b_train, y_b_train)
y_hat_skl = skl_clf.predict(X_b_test)
print('sklearn LogRegression:')
print('mean', y_hat_skl.mean())
print('accuracy', accuracy_score(y_b_test, y_hat_skl))
print('ROC-AUC: ',roc_auc_score(y_b_test, skl_clf.predict_proba(X_b_test)[:,0]))


sklearn LogRegression:
mean 0.9232
accuracy 0.5184
ROC-AUC:  0.48610748108972385


On this dataset our custom regression is also better than the original one(by looking at the ROC-AUC score).

Lets try to upsample the training dataset by adding copies of the samples with 1:2 ratio in order to prevent the unbalanced issue:

In [ ]:
def upsample(X, y, repeat):
    X_0 = X[y == 0]
    X_1 = X[y == 1]
    y_0 = y[y == 0]
    y_1 = y[y == 1]

    X_upsampled = np.concatenate([X_0] + [X_1] * repeat)
    y_upsampled = np.concatenate([y_0] + [y_1] * repeat)
    
    X_upsampled, y_upsampled = shuffle(X_upsampled, y_upsampled, random_state=42)
    
    return X_upsampled, y_upsampled

In [ ]:
X_train_up, y_train_up = upsample(X_b_train, y_b_train, 2)

print('Upsampled train features', X_train_up.shape)
print('Upsampled train target', y_train_up.shape)
print()
print('Upsampled target mean',y_train_up.mean() )

Upsampled train features (11363, 11)
Upsampled train target (11363,)

Upsampled target mean 0.6799260758602482


Now the balance is way better than the original dataset.

In [ ]:
clf.fit(X_train_up, y_train_up, iters = 20)

Iteration: 1, Loss: 9009.806887299681
Iteration: 2, Loss: 7094.168204901615
Iteration: 3, Loss: 6916.8125788202415
Iteration: 4, Loss: 6903.375104030491
Iteration: 5, Loss: 6902.088090168179
Iteration: 6, Loss: 6901.88430023991
Iteration: 7, Loss: 6901.827503179093
Iteration: 8, Loss: 6901.805216555675
Iteration: 9, Loss: 6901.7949555227715
Iteration: 10, Loss: 6901.789830023607
Iteration: 11, Loss: 6901.787147436089
Iteration: 12, Loss: 6901.785703502352
Iteration: 13, Loss: 6901.784912885452
Iteration: 14, Loss: 6901.784475409504
Iteration: 15, Loss: 6901.784231750018
Iteration: 16, Loss: 6901.784095479137
Iteration: 17, Loss: 6901.784019065506
Iteration: 18, Loss: 6901.783976142492
Iteration: 19, Loss: 6901.783952003796


In [ ]:
y_hat_lr = clf.predict(X_b_test)
print('LogRegression from scratch:')
print('mean', y_hat_lr.mean())
print('accuracy', accuracy_score(y_b_test, y_hat_lr))
print('ROC-AUC: ', roc_auc_score(y_b_test, clf.predict_proba(X_b_test)))

LogRegression from scratch:
mean 0.9276
accuracy 0.5324
ROC-AUC:  0.5999900067646516


In [ ]:
skl_clf = LogisticRegression()

skl_clf.fit(X_train_up, y_train_up)
y_hat_skl = skl_clf.predict(X_b_test)
print('sklearn LogRegression:')
print('mean', y_hat_skl.mean())
print('accuracy', accuracy_score(y_b_test, y_hat_skl))
print('ROC-AUC: ',roc_auc_score(y_b_test, skl_clf.predict_proba(X_b_test)[:,0]))

sklearn LogRegression:
mean 1.0
accuracy 0.5152
ROC-AUC:  0.48390960990508985


As we can see there are almost no impact to the metrics after upsampling. Bear in mind the test dataset has the same inbalance as the original one.

## Summary




Even considering the fact than the default sklearn Logistic regression uses different optimisation method and we haven't tried to fine tune it by using different hyperparameters and trying to decrease the imbalance impact, its not a bad result.

However that wasn't the main goal of this work. The main goal was a better understanding of the ML algorythms and to see how it works. From my point of view this goal is achieved. 
Hope you enjoy that!